# Quantum Conversations: Comprehensive Demo

This notebook demonstrates the complete Quantum Conversations toolkit, including particle filter generation, multiple visualization types, and analysis tools.

In [ ]:
import sys
sys.path.append('../')

from quantum_conversations import (
    ParticleFilter, 
    TokenSequenceVisualizer,
    ModelManager,
    save_particles,
    load_particles,
    compute_sequence_entropy,
    compute_divergence_score,
    create_probability_tensor,
    get_top_tokens
)
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Create output directory
output_dir = Path('../outputs/demo_figures')
output_dir.mkdir(parents=True, exist_ok=True)

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## 1. Model Management

Demonstrate the flexible model management system that supports any open-weights HuggingFace model.

In [ ]:
# Initialize model manager
model_manager = ModelManager()

# Load different models to show flexibility
models_to_test = [
    "EleutherAI/pythia-70m",  # Small model for quick testing
    "gpt2",                    # Classic GPT-2
]

for model_name in models_to_test:
    print(f"\nLoading {model_name}...")
    model, tokenizer = model_manager.load_model(model_name, device="cpu")
    print(f"  ✓ Model loaded: {model.__class__.__name__}")
    print(f"  ✓ Vocab size: {tokenizer.vocab_size}")
    print(f"  ✓ Cached: {model_name in model_manager.model_cache}")

## 2. Particle Filter Generation

Generate particles with different configurations to explore the token probability space.

In [ ]:
# Test different prompts and temperatures
test_configs = [
    {"prompt": "The future of AI is", "temp": 0.5, "label": "Low Temperature (0.5)"},
    {"prompt": "The future of AI is", "temp": 1.0, "label": "Medium Temperature (1.0)"},
    {"prompt": "The future of AI is", "temp": 1.5, "label": "High Temperature (1.5)"},
]

all_particles = {}

for config in test_configs:
    pf = ParticleFilter(
        model_name="EleutherAI/pythia-70m",
        n_particles=8,
        temperature=config["temp"],
        device="cpu",
        model_manager=model_manager,
        seed=42
    )
    
    particles = pf.generate(config["prompt"], max_new_tokens=15)
    all_particles[config["label"]] = (particles, pf.tokenizer)
    
    print(f"\n{config['label']}:")
    print(f"  Generated {len(particles)} particles")
    print(f"  Sample outputs:")
    for i, p in enumerate(particles[:3]):
        text = pf.tokenizer.decode(p.token_ids, skip_special_tokens=True)
        print(f"    {i+1}. {text[:60]}...")

## 3. Visualization Suite

### 3.1 Bumplot Visualization

In [ ]:
# Create bumplot for each temperature setting
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

for idx, (label, (particles, tokenizer)) in enumerate(all_particles.items()):
    viz = TokenSequenceVisualizer(tokenizer=tokenizer)
    
    # Create bumplot in subplot
    plt.sca(axes[idx])
    viz.visualize_bumplot(
        particles,
        color_by='transition_prob',
        max_vocab_display=25,
        show_tokens=False,
        figsize=None  # Use existing figure
    )
    axes[idx].set_title(f"Bumplot: {label}")

plt.suptitle("Token Trajectory Comparison Across Temperatures", fontsize=16, y=1.02)
plt.tight_layout()
plt.savefig(output_dir / "bumplot_temperature_comparison.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\nSaved: {output_dir / 'bumplot_temperature_comparison.png'}")

### 3.2 Sankey Diagram Visualization

In [ ]:
# Generate particles for Sankey visualization
pf_sankey = ParticleFilter(
    model_name="EleutherAI/pythia-70m",
    n_particles=10,
    temperature=1.0,
    device="cpu",
    model_manager=model_manager,
    seed=42
)

prompt_sankey = "Once upon a time"
particles_sankey = pf_sankey.generate(prompt_sankey, max_new_tokens=12)

viz_sankey = TokenSequenceVisualizer(tokenizer=pf_sankey.tokenizer)
fig = viz_sankey.visualize(
    particles_sankey,
    prompt_sankey,
    figsize=(14, 8),
    output_path=str(output_dir / "sankey_diagram.png")
)
plt.show()

print(f"\nSaved: {output_dir / 'sankey_diagram.png'}")

### 3.3 Probability Heatmap

In [ ]:
# Create probability heatmap
fig = viz_sankey.visualize_heatmap(
    particles_sankey,
    figsize=(12, 8),
    output_path=str(output_dir / "probability_heatmap.png")
)
plt.show()

print(f"\nSaved: {output_dir / 'probability_heatmap.png'}")

## 4. Analysis Tools

### 4.1 Entropy and Divergence Analysis

In [ ]:
# Analyze entropy and divergence for different temperatures
analysis_results = {}

for label, (particles, tokenizer) in all_particles.items():
    entropies = [compute_sequence_entropy(p) for p in particles]
    divergence = compute_divergence_score(particles)
    
    analysis_results[label] = {
        'mean_entropy': np.mean(entropies),
        'std_entropy': np.std(entropies),
        'divergence': divergence
    }

# Plot analysis results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Entropy plot
labels = list(analysis_results.keys())
mean_entropies = [analysis_results[l]['mean_entropy'] for l in labels]
std_entropies = [analysis_results[l]['std_entropy'] for l in labels]

x_pos = np.arange(len(labels))
ax1.bar(x_pos, mean_entropies, yerr=std_entropies, capsize=10, alpha=0.7)
ax1.set_xticks(x_pos)
ax1.set_xticklabels(labels, rotation=45, ha='right')
ax1.set_ylabel('Mean Entropy')
ax1.set_title('Sequence Entropy by Temperature')
ax1.grid(True, alpha=0.3)

# Divergence plot
divergences = [analysis_results[l]['divergence'] for l in labels]
ax2.bar(x_pos, divergences, alpha=0.7, color='orange')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(labels, rotation=45, ha='right')
ax2.set_ylabel('Divergence Score')
ax2.set_title('Path Divergence by Temperature')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / "entropy_divergence_analysis.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\nSaved: {output_dir / 'entropy_divergence_analysis.png'}")
print("\nAnalysis Results:")
for label, results in analysis_results.items():
    print(f"\n{label}:")
    print(f"  Mean Entropy: {results['mean_entropy']:.4f} ± {results['std_entropy']:.4f}")
    print(f"  Divergence Score: {results['divergence']:.4f}")

### 4.2 Probability Tensor Analysis

In [ ]:
# Create and analyze probability tensor
particles_for_tensor, tokenizer_for_tensor = all_particles["Medium Temperature (1.0)"]

# Create probability tensor
tensor = create_probability_tensor(particles_for_tensor)
print(f"Probability tensor shape: {tensor.shape}")
print(f"  (vocab_size, timesteps, n_particles) = {tensor.shape}")

# Get top tokens at each timestep
for t in range(min(5, tensor.shape[1])):
    top_tokens = get_top_tokens(particles_for_tensor, timestep=t, k=5)
    print(f"\nTimestep {t} - Top 5 tokens:")
    for token_id, prob in top_tokens:
        token_text = tokenizer_for_tensor.decode([token_id])
        print(f"  '{token_text}': {prob:.4f}")

## 5. Persistence: Save and Load Particles

In [ ]:
# Save particles
save_path = output_dir / "saved_particles.pkl"
particles_to_save, tokenizer_to_save = all_particles["High Temperature (1.5)"]
save_particles(particles_to_save, save_path)
print(f"Saved {len(particles_to_save)} particles to {save_path}")

# Load particles
loaded_particles = load_particles(save_path)
print(f"\nLoaded {len(loaded_particles)} particles from {save_path}")

# Verify loaded particles
print("\nVerification:")
print(f"  Original first particle tokens: {particles_to_save[0].token_ids[:5]}")
print(f"  Loaded first particle tokens: {loaded_particles[0].token_ids[:5]}")
print(f"  ✓ Particles successfully saved and loaded" if 
      particles_to_save[0].token_ids == loaded_particles[0].token_ids else 
      "  ✗ Error in save/load")

## 6. Different Model Sizes Comparison

In [ ]:
# Compare different model sizes
models_comparison = {
    "Pythia-70M": "EleutherAI/pythia-70m",
    "GPT-2 (124M)": "gpt2"
}

prompt_comparison = "The secret to happiness is"
comparison_particles = {}

for model_label, model_name in models_comparison.items():
    print(f"\nGenerating with {model_label}...")
    pf = ParticleFilter(
        model_name=model_name,
        n_particles=6,
        temperature=1.0,
        device="cpu",
        model_manager=model_manager,
        seed=42
    )
    
    particles = pf.generate(prompt_comparison, max_new_tokens=15)
    comparison_particles[model_label] = (particles, pf.tokenizer)
    
    # Show sample outputs
    print(f"  Sample outputs from {model_label}:")
    for i, p in enumerate(particles[:2]):
        text = pf.tokenizer.decode(p.token_ids, skip_special_tokens=True)
        print(f"    {text[:80]}")

In [ ]:
# Visualize model comparison with bumplots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for idx, (model_label, (particles, tokenizer)) in enumerate(comparison_particles.items()):
    viz = TokenSequenceVisualizer(tokenizer=tokenizer)
    plt.sca(axes[idx])
    viz.visualize_bumplot(
        particles,
        color_by='particle_id',
        max_vocab_display=20,
        show_tokens=False,
        figsize=None
    )
    axes[idx].set_title(f"{model_label}\nPrompt: '{prompt_comparison}'")

plt.tight_layout()
plt.savefig(output_dir / "model_comparison_bumplot.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"\nSaved: {output_dir / 'model_comparison_bumplot.png'}")

## 7. Interactive Exploration

Try your own prompts and configurations!

In [ ]:
# Interactive generation function
def explore_prompt(prompt, model_name="EleutherAI/pythia-70m", n_particles=8, 
                   temperature=1.0, max_tokens=15):
    """Generate and visualize particles for a custom prompt."""
    
    print(f"\nExploring: '{prompt}'")
    print(f"Settings: model={model_name}, particles={n_particles}, temp={temperature}")
    print("="*60)
    
    # Generate particles
    pf = ParticleFilter(
        model_name=model_name,
        n_particles=n_particles,
        temperature=temperature,
        device="cpu",
        model_manager=model_manager,
        seed=None  # Random seed for variety
    )
    
    particles = pf.generate(prompt, max_new_tokens=max_tokens)
    
    # Show outputs
    print("\nGenerated sequences:")
    for i, p in enumerate(particles):
        text = pf.tokenizer.decode(p.token_ids, skip_special_tokens=True)
        print(f"  {i+1}. {text}")
    
    # Create visualizations
    viz = TokenSequenceVisualizer(tokenizer=pf.tokenizer)
    
    # Bumplot
    fig = viz.visualize_bumplot(
        particles,
        color_by='transition_prob',
        max_vocab_display=30,
        show_tokens=False,
        figsize=(14, 6)
    )
    plt.title(f"Token Trajectories: '{prompt}'")
    plt.tight_layout()
    plt.show()
    
    # Compute metrics
    divergence = compute_divergence_score(particles)
    entropies = [compute_sequence_entropy(p) for p in particles]
    
    print(f"\nMetrics:")
    print(f"  Divergence score: {divergence:.4f}")
    print(f"  Mean entropy: {np.mean(entropies):.4f}")
    
    return particles

# Example usage
custom_particles = explore_prompt(
    "In a world where",
    temperature=1.2,
    n_particles=8,
    max_tokens=20
)

In [ ]:
# Try different prompt types
interesting_prompts = [
    "The most important thing in life is",
    "def fibonacci(n):",
    "Breaking news:",
    "To be or not to be,"
]

for prompt in interesting_prompts:
    particles = explore_prompt(prompt, temperature=1.0, n_particles=6, max_tokens=12)

## Summary

This comprehensive demo showcased:

1. **Model Management**: Flexible loading of any open-weights HuggingFace model
2. **Particle Generation**: Multiple hypothesis tracking with configurable parameters
3. **Visualization Suite**:
   - Bumplot visualization for token trajectories
   - Sankey diagrams for path visualization
   - Probability heatmaps
4. **Analysis Tools**: Entropy and divergence metrics
5. **Probability Tensors**: V×t×n tensor creation and analysis
6. **Persistence**: Save/load functionality for particles
7. **Model Comparison**: Comparing behavior across different model sizes
8. **Interactive Exploration**: Custom prompt analysis

All visualizations and outputs have been saved to: `code/outputs/demo_figures/`